In [1]:
# Cell 1: scripts folder to path & import modules for whole coding
import sys
from pathlib import Path
import time
from datetime import timedelta
# 1) Locate project root (one level up from this notebooks directory)
try:
    project_root = Path(__file__).resolve().parent.parent
except NameError:
    project_root = Path.cwd().parent
    
print(f"Project root identified as: {project_root}")
# 2) Point at your scripts folder
scripts_dir = project_root / "5_nsga" / "5_nsga_scripts"
assert scripts_dir.exists(), f"Can't find scripts at {scripts_dir}"
sys.path.insert(0, str(scripts_dir))
# 3) Import helper modules
from pv_simulate_30 import simulate_multi_year_pv
from battery_30 import simulate_battery_dispatch
from financial_30 import compute_financials
from objective_extra_pv import evaluate_solution  # Import from the new module
import objective_extra_pv  # Import as module to set global variables
# 4) Import optimiser & data libs
import numpy as np
import pandas as pd
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.core.callback import Callback
print("All modules imported successfully - can move onto next stage")
print("Using battery + additional PV optimization")

Project root identified as: /Users/petertunali/Documents/GitHub/Battery_Optimisation
All modules imported successfully - can move onto next stage
Using battery + additional PV optimization


In [2]:
# Cell 2: Configure outputs, check data & build 30‑yr demand
from pathlib import Path
import pandas as pd
# 1) Auto‑version your outputs folder under project_root/outputs_optimisation
base_out = project_root / "5_nsga_outputs"
base_out.mkdir(exist_ok=True)

# Update the folder naming pattern
existing = [d.name for d in base_out.iterdir() if d.is_dir() and 
           (d.name.isdigit() or "extra_pv" in d.name)]
extra_pv_folders = [d for d in existing if "extra_pv" in d]  # Remove the .name since existing is already names
if extra_pv_folders:
    # Extract numbers from folder names like "001_extra_pv"
    nums = sorted([int(d.split('_')[0]) for d in extra_pv_folders])  # Remove the .name since extra_pv_folders contains strings
    next_run = nums[-1] + 1 if nums else 1
else:
    # If no extra_pv folders exist, look at regular numeric folders
    nums = sorted([int(n) for n in existing if n.isdigit()]) if existing else []
    next_run = nums[-1] + 1 if nums else 1

# Create a new folder with the extra_pv suffix
out_dir = base_out / f"{next_run:03d}_extra_pv"
out_dir.mkdir()
print(f"Writing all outputs to → {out_dir}")

# 2) Locate your data directory
data_dir = project_root / "data"
if not data_dir.exists():
    for alt in [project_root/"Battery_Optimisation"/"data", Path.cwd()/"data"]:
        if alt.exists():
            data_dir = alt
            break
print(f"Using data directory → {data_dir}")

# 3) Check EPW weather files
weather_files = [
    str(data_dir / "Bonfire_2025.epw"),
    str(data_dir / "Bonfire_2040_4_5.epw"),
    str(data_dir / "Bonfire_2050_4_5.epw")
]
print("\nChecking weather files:")
for wf in weather_files:
    print(" ✔ Found:" if Path(wf).exists() else " ❌ Missing:", wf)
assert all(Path(wf).exists() for wf in weather_files), "One or more weather files missing!"

# 4) Find demand CSV (take the first match)
demand_paths = [
    data_dir / "PV_Generation_excel.csv",
    data_dir / "Energy_Demand_and_Supply_2024.csv"
]
for p in demand_paths:
    if p.exists():
        demand_file = p
        print("\n✔ Found demand file:", p)
        break
else:
    raise FileNotFoundError("No demand file found – check your filenames")

# 5) Simplified demand load function
def load_demand_profile(csv_path: Path) -> pd.Series:
    # Load the CSV and skip NaN timestamp rows immediately
    raw = pd.read_csv(csv_path, parse_dates=['Date and Time'], dayfirst=True)
    
    # Drop rows with NaN timestamps right away
    raw = raw.dropna(subset=['Date and Time'])
    print(f"CSV loaded with {len(raw)} valid rows")
    
    # Get the consumption column
    consumption_col = 'Consumtpion (kWh)'  # Based on your data sample
    if consumption_col not in raw.columns:
        # Try to find the consumption column
        for col in raw.columns:
            if 'consum' in col.lower() or 'demand' in col.lower():
                consumption_col = col
                break
        else:
            # If no consumption column found, use the second column
            consumption_col = raw.columns[1]
    
    print(f"Using consumption column: '{consumption_col}'")
    
    # Create a Series with timestamp index and consumption values
    s = pd.Series(raw[consumption_col].values, index=raw['Date and Time'])
    
    # Check for duplicate timestamps
    dup_count = s.index.duplicated().sum()
    if dup_count:
        print(f"⚠️ Dropping {dup_count} duplicate timestamps")
        s = s[~s.index.duplicated(keep='first')]
    
    # Build the expected half-hour index for entire year (no Feb 29)
    year = s.index.min().year
    start = pd.Timestamp(year, 1, 1, 0, 0)
    end = pd.Timestamp(year, 12, 31, 23, 30)
    expected = pd.date_range(start, end, freq="30min")
    expected = expected[~((expected.month==2) & (expected.day==29))]
    
    # Reindex to ensure complete coverage
    s = s.reindex(expected)
    missing = s.isna().sum()
    if missing:
        print(f"⚠️ Filling {missing} missing points with 0")
        s = s.fillna(0.0)
    
    # Final sanity check
    assert len(s) == 17520, f"Got {len(s)} points, expected 17520"
    return s

# 6) Load one-year demand data and build 30-year profile
print(f"Loading demand data from: {demand_file}\n")
one_year_dem = load_demand_profile(demand_file)
print(f"One year data points: {len(one_year_dem)}")

# 7) Create 30-year demand profile with proper date handling
start_year = one_year_dem.index[0].year
years = 30
all_data = []

# Build 30 years of data year by year
for year_offset in range(years):
    # Copy the data for this year
    year_data = one_year_dem.copy()
    
    # Create index for this specific year
    year_start = pd.Timestamp(start_year + year_offset, 1, 1, 0, 0)
    year_end = pd.Timestamp(start_year + year_offset, 12, 31, 23, 30)
    year_range = pd.date_range(start=year_start, end=year_end, freq="30min")
    
    # Remove Feb 29 if it's a leap year
    year_range = year_range[~((year_range.month == 2) & (year_range.day == 29))]
    
    # Make sure it has the right number of points
    assert len(year_range) == len(one_year_dem), f"Year {start_year + year_offset} has {len(year_range)} points, expected {len(one_year_dem)}"
    
    # Assign the new index and add to our list
    year_data.index = year_range
    all_data.append(year_data)

# Concatenate all years
demand_profile = pd.concat(all_data)
print("\n30‑year demand profile built:")
print(f"  • Time steps : {len(demand_profile)}")
print(f"  • Date range : {demand_profile.index[0]} → {demand_profile.index[-1]}")
print(f"  • Total demand: {demand_profile.sum():.2f} kWh")

Writing all outputs to → /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga_outputs/011_extra_pv
Using data directory → /Users/petertunali/Documents/GitHub/Battery_Optimisation/data

Checking weather files:
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2025.epw
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2040_4_5.epw
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2050_4_5.epw

✔ Found demand file: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv
Loading demand data from: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv

CSV loaded with 17520 valid rows
Using consumption column: 'Consumtpion (kWh)'
One year data points: 17520

30‑year demand profile built:
  • Time steps : 525600
  • Date range : 2025-01-01 00:00:00 → 2054-12-31 23:30:00
  • Total demand: 1318609.49 kWh


In [3]:
# Cell 3: Define the PV systems and roof parameters
# Define PV system parameters - keep your original 10kW configuration
roof_params = [{
    'name': 'main_roof',
    'system_capacity_kw': 10.0,  # 10kW system
    'tilt': 10.0,
    'azimuth': 18.0,  # North-facing in Southern Hemisphere
    'shading': 43.0,  # 43% shading impact
}]
print("Roof parameters:")
for roof in roof_params:
    print(f"  - {roof['name']}: {roof['system_capacity_kw']} kW, Tilt: {roof['tilt']}°, Azimuth: {roof['azimuth']}°, Shading: {roof.get('shading', 0.0)}%")
total_pv_capacity = sum(roof['system_capacity_kw'] for roof in roof_params)
print(f"Total PV capacity: {total_pv_capacity} kW")
print("ready for next stage")

Roof parameters:
  - main_roof: 10.0 kW, Tilt: 10.0°, Azimuth: 18.0°, Shading: 43.0%
Total PV capacity: 10.0 kW
ready for next stage


In [4]:
# Cell 4: Build 30‑yr PV & demand, then compute zero‑battery baseline
print("1) Simulating 30 yr PV…")
start_time = time.time()
start_years = [2025, 2040, 2050]
pv_profile = simulate_multi_year_pv(
    weather_files    = weather_files,
    roof_params      = roof_params,
    repeats_per_file = 10,
    start_years      = start_years
)
elapsed = time.time() - start_time
print(f"PV simulation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")
print(f"   • PV steps : {len(pv_profile)}")
print(f"   • Date range: {pv_profile.index[0]} → {pv_profile.index[-1]}")
print(f"   • Gen total : {pv_profile['simulated_kwh'].sum():.2f} kWh\n")

# Cell 4: Build 30‑yr PV & demand, then compute baselines
print("1) Simulating 30 yr PV for existing system…")
start_time = time.time()
start_years = [2025, 2040, 2050]
pv_profile = simulate_multi_year_pv(
    weather_files=weather_files,
    roof_params=roof_params,
    repeats_per_file=10,
    start_years=start_years
)
elapsed = time.time() - start_time
print(f"PV simulation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")
print(f"   • PV steps: {len(pv_profile)}")
print(f"   • Date range: {pv_profile.index[0]} → {pv_profile.index[-1]}")
print(f"   • Total generation: {pv_profile['simulated_kwh'].sum():.2f} kWh\n")

# 4b) Compute baselines for both no-PV and PV-only scenarios
print("Computing baseline costs...")
start_time = time.time()

# Define actual electricity costs from your electricity bill data
ANNUAL_NO_PV_COST = 9424.48      # From your bill - no PV, no battery
ANNUAL_PV_ONLY_COST = 8246.44    # From your bill - with PV, no battery

# Calculate 30-year baseline costs
NO_PV_BASELINE_COST = ANNUAL_NO_PV_COST * 30
PV_ONLY_BASELINE_COST = ANNUAL_PV_ONLY_COST * 30

print(f"Annual no-PV cost: ${ANNUAL_NO_PV_COST:,.2f}")
print(f"Annual PV-only cost: ${ANNUAL_PV_ONLY_COST:,.2f}")
print(f"Annual savings from PV alone: ${ANNUAL_NO_PV_COST - ANNUAL_PV_ONLY_COST:,.2f}")
print(f"30-year no-PV baseline: ${NO_PV_BASELINE_COST:,.2f}")
print(f"30-year PV-only baseline: ${PV_ONLY_BASELINE_COST:,.2f}")

# Simulate with existing PV, no battery to verify
disp0, totals0 = simulate_battery_dispatch(
    pv_gen=pv_profile['simulated_kwh'],
    demand=demand_profile,
    battery_kwh=0.0,
    roundtrip_eff=0.9,
    min_soc_pct=0.05,
    annual_deg_rate=0.01,
    grid_emission_rate=0.81
)

# Calculate PV-only annual cost from simulation
simulated_pv_only_import_cost = (
    totals0['total_grid_import_peak'] * 0.39710 + 
    totals0['total_grid_import_offpeak'] * 0.13530
) / 30
simulated_pv_only_export_rev = totals0['total_pv_export'] * 0.033 / 30
SIMULATED_PV_ONLY_ANNUAL_COST = simulated_pv_only_import_cost - simulated_pv_only_export_rev
SIMULATED_PV_ONLY_TOTAL_COST = (simulated_pv_only_import_cost - simulated_pv_only_export_rev) * 30

print(f"\nSimulated annual PV-only cost: ${SIMULATED_PV_ONLY_ANNUAL_COST:,.2f}")
print(f"Actual annual PV-only cost from bill: ${ANNUAL_PV_ONLY_COST:,.2f}")
print(f"Difference: ${SIMULATED_PV_ONLY_ANNUAL_COST - ANNUAL_PV_ONLY_COST:,.2f}")

# For optimization, we'll use the actual values from the bill for better accuracy
BASE_IMPORT_COST = NO_PV_BASELINE_COST       # No PV, no battery
PV_ONLY_COST = PV_ONLY_BASELINE_COST         # PV-only, no battery

# Set all needed values in the objective_extra_pv module
objective_extra_pv.BASE_IMPORT_COST = BASE_IMPORT_COST
objective_extra_pv.PV_ONLY_COST = PV_ONLY_COST
objective_extra_pv.WEATHER_FILES = weather_files
objective_extra_pv.START_YEARS = start_years
objective_extra_pv.EXISTING_PV_CAPACITY = total_pv_capacity

print(f"\n   → Baseline no-PV cost over 30 yr = ${BASE_IMPORT_COST:,.2f}")
print(f"   → Baseline PV-only cost over 30 yr = ${PV_ONLY_COST:,.2f}")

elapsed = time.time() - start_time
print(f"Baseline calculation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")

1) Simulating 30 yr PV…
PV simulation completed in 0.2 seconds (0:00:00)
   • PV steps : 525600
   • Date range: 2025-01-01 00:00:00 → 2054-12-24 23:30:00
   • Gen total : 204028.14 kWh

1) Simulating 30 yr PV for existing system…
PV simulation completed in 0.1 seconds (0:00:00)
   • PV steps: 525600
   • Date range: 2025-01-01 00:00:00 → 2054-12-24 23:30:00
   • Total generation: 204028.14 kWh

Computing baseline costs...
Annual no-PV cost: $9,424.48
Annual PV-only cost: $8,246.44
Annual savings from PV alone: $1,178.04
30-year no-PV baseline: $282,734.40
30-year PV-only baseline: $247,393.20

Simulated annual PV-only cost: $8,249.58
Actual annual PV-only cost from bill: $8,246.44
Difference: $3.14

   → Baseline no-PV cost over 30 yr = $282,734.40
   → Baseline PV-only cost over 30 yr = $247,393.20
Baseline calculation completed in 1.2 seconds (0:00:01)


In [5]:
# Debug Cell: Analyze a specific solution from the Pareto front
# Enter the values manually from the optimization results
battery_kwh = 15.0  # Adjust to analyze a specific solution
additional_pv_kw = 30.0  # Adjust to analyze a specific solution
allocation_factor = 0.3  # Adjust to analyze a specific solution

print(f"Analyzing solution: {battery_kwh} kWh battery, {additional_pv_kw} kW additional PV, {allocation_factor*100:.0f}% allocation to existing roof")

# Calculate the allocation of additional PV capacity
type_a_capacity = min(additional_pv_kw * allocation_factor, 26.40)  # Capped at max capacity
type_b_capacity = additional_pv_kw - type_a_capacity

print(f"Type A (existing roof): {type_a_capacity:.2f} kW")
print(f"Type B (optimal angle): {type_b_capacity:.2f} kW")

# Create roof parameters for simulation
additional_roof_params = []

# Add Type A if capacity > 0
if type_a_capacity > 0:
    type_a_params = {
        'name': 'existing_roof_extension',
        'system_capacity_kw': type_a_capacity,
        'tilt': 12.5,
        'azimuth': 37.0,
        'shading': 6.18,
        'array_type': 1,  # Fixed roof mount
    }
    additional_roof_params.append(type_a_params)

# Add Type B if capacity > 0
if type_b_capacity > 0:
    type_b_params = {
        'name': 'optimal_angle_installation',
        'system_capacity_kw': type_b_capacity,
        'tilt': 30.0,
        'azimuth': 5.0,
        'shading': 0.0,
        'array_type': 0,  # Fixed open rack
    }
    additional_roof_params.append(type_b_params)

# Simulate additional PV generation
additional_pv_profile = simulate_multi_year_pv(
    weather_files=weather_files,
    roof_params=additional_roof_params,
    repeats_per_file=10,
    start_years=start_years
)

# Combine existing and additional PV generation
total_pv_kwh = pv_profile['simulated_kwh'] + additional_pv_profile['simulated_kwh']
total_pv_profile = pd.DataFrame({'simulated_kwh': total_pv_kwh})

# Update total PV capacity
total_pv_capacity_new = total_pv_capacity + additional_pv_kw

# Simulate battery dispatch
disp, totals = simulate_battery_dispatch(
    pv_gen=total_pv_profile['simulated_kwh'],
    demand=demand_profile,
    battery_kwh=battery_kwh,
    battery_kw=battery_kwh * 0.5,  # 0.5C power rating
    roundtrip_eff=0.9,             # 90% round trip efficiency
    min_soc_pct=0.05,              # 95% depth of discharge (5% min SOC)
    annual_deg_rate=0.01,          # 1% degradation fallback rate
    grid_emission_rate=0.81        # 0.81 kg CO2e/kWh
)

# Calculate PV capital costs
pv_cost_per_kw = 1500.0  # Base cost per kW for PV
pv_type_a_cost = type_a_capacity * pv_cost_per_kw  # Existing roof installation
pv_type_b_cost = type_b_capacity * (pv_cost_per_kw * 1.15)  # 15% premium for optimal-angle installation
pv_installation_cost = 1000.0 if additional_pv_kw > 0 else 0.0  # Fixed installation overhead
total_pv_cost = pv_type_a_cost + pv_type_b_cost + pv_installation_cost

# Calculate battery cost using formula with min $600/kWh
if battery_kwh <= 0:
    battery_cost_per_kwh = 0.0  # No cost for 0 kWh battery
else:
    battery_cost_per_kwh = 977.54 * np.exp(-0.004 * battery_kwh)
    battery_cost_per_kwh = max(600.0, battery_cost_per_kwh)  # Minimum $600/kWh

print(f"Battery cost: ${battery_cost_per_kwh:.2f} per kWh")
print(f"Total PV cost: ${total_pv_cost:,.2f}")

# Compute financial metrics
fin = compute_financials(
    totals,
    battery_kwh=battery_kwh,
    pv_kw=total_pv_capacity_new,                 
    pv_cost_per_kw=total_pv_cost/additional_pv_kw if additional_pv_kw > 0 else 0.0,  # Effective cost per kW
    pv_installation_cost=0.0,              # Already included in total_pv_cost
    battery_cost_per_kwh=battery_cost_per_kwh,
    battery_installation_cost_per_kwh=174.0,
    battery_power_ratio=0.5,               # Power rating as fraction of capacity
    pv_maintenance_per_kw_day=0.13,        # $0.13 per kW per day for PV
    battery_maintenance_per_kw_day=0.12,   # $0.12 per kW per day for battery
    discount_rate=0.07,
    baseline_import_cost=BASE_IMPORT_COST,  # No PV, no battery cost
    baseline_pv_only_cost=PV_ONLY_COST      # PV-only, no battery cost
)

# Extract and display key metrics
print("\nFinancial Results:")
print(f"IRR: {fin['irr']*100 if fin['irr'] is not None else 'N/A':.2f}%")
print(f"NPV: ${-fin['net_cost']:,.2f}")
print(f"Total Capital Cost: ${fin['capex_total']:,.2f}")
print(f"Annual Maintenance: ${fin['opex_total']/30:,.2f}/year")

print("\nEnergy Results:")
print(f"Total Generation: {total_pv_profile['simulated_kwh'].sum():,.2f} kWh")
print(f"Total Demand: {totals['total_demand']:,.2f} kWh")
print(f"Self-Consumption Rate: {totals['self_consumption_rate']*100:.2f}%")
print(f"Renewable Fraction: {totals['renewable_fraction']*100:.2f}%")
print(f"Grid Imports: {totals['total_grid_import_peak'] + totals['total_grid_import_offpeak']:,.2f} kWh")
print(f"Grid Exports: {totals['total_pv_export']:,.2f} kWh")
print(f"CO2 Emissions: {totals['total_grid_emissions']:,.2f} kg CO2e")

# Calculate annual electricity bill
annual_import_cost = (
    totals['total_grid_import_peak'] * 0.39710 + 
    totals['total_grid_import_offpeak'] * 0.13530
) / 30
annual_export_revenue = totals['total_pv_export'] * 0.033 / 30
annual_bill = annual_import_cost - annual_export_revenue

print(f"\nAnnual Electricity Bill: ${annual_bill:,.2f}/year")
print(f"Savings vs. Baseline (no PV): ${ANNUAL_NO_PV_COST - annual_bill:,.2f}/year")
print(f"Savings vs. Existing PV only: ${ANNUAL_PV_ONLY_COST - annual_bill:,.2f}/year")

Analyzing solution: 15.0 kWh battery, 30.0 kW additional PV, 30% allocation to existing roof
Type A (existing roof): 9.00 kW
Type B (optimal angle): 21.00 kW
Battery cost: $920.61 per kWh
Total PV cost: $50,725.00

Financial Results:
IRR: 2.75%
NPV: $40,455.50
Total Capital Cost: $84,052.52
Annual Maintenance: $2,226.50/year

Energy Results:
Total Generation: 1,441,320.42 kWh
Total Demand: 1,316,358.47 kWh
Self-Consumption Rate: 44.34%
Renewable Fraction: 53.10%
Grid Imports: 617,346.71 kWh
Grid Exports: 727,992.66 kWh
CO2 Emissions: 500,050.84 kg CO2e

Annual Electricity Bill: $3,047.71/year
Savings vs. Baseline (no PV): $6,376.77/year
Savings vs. Existing PV only: $5,198.73/year


In [6]:
# Cell 6: Define the optimisation problem to solve
print("Defining optimization problem with battery + additional PV (IRR and NPV only)...")
try:
    # Define the NSGA-II optimization problem
    class BatteryPVOptimizationProblem(Problem):
        def __init__(self):
            # Define three variables:
            # 1. Battery size (kWh) - 0 to 50 kWh
            # 2. Additional PV capacity (kW) - 0 to 50 kW
            # 3. Allocation factor (%) - 0 to 1
            super().__init__(
                n_var=3,               # Number of decision variables
                n_obj=2,               # Number of objectives (IRR, NPV) - reduced from 3
                n_ieq_constr=0,        # Number of inequality constraints
                xl=np.array([0.0, 0.0, 0.0]),  # Lower bounds
                xu=np.array([50.0, 50.0, 1.0]) # Upper bounds
            )
            # Store the profiles for use in evaluation
            self.pv_profile = pv_profile
            self.demand_profile = demand_profile
        
        def _evaluate(self, x, out, *args, **kwargs):
            # Evaluate each solution in the populationA
            n_solutions = x.shape[0]
            F = np.zeros((n_solutions, 2))  # For storing [-IRR, -NPV] only
            
            # Loop through each solution
            for i in range(n_solutions):
                battery_kwh = x[i, 0]
                additional_pv_kw = x[i, 1]
                allocation_factor = x[i, 2]
                
                # Define parameters for evaluation
                params = {
                    'battery_kwh': battery_kwh,
                    'additional_pv_kw': additional_pv_kw,
                    'allocation_factor': allocation_factor
                }
                
                # Get the evaluation results (now we only need IRR and NPV)
                result = evaluate_solution_financial_only(params, self.pv_profile, self.demand_profile)
                F[i, :] = result
            
            # Set the output objectives array
            out["F"] = F
    
    # Define a new function for financial-only evaluation
    def evaluate_solution_financial_only(params, pv_profile, demand_profile):
        """Modified evaluate_solution that returns only financial metrics (IRR, NPV)"""
        # Extract parameters
        battery_kwh = params['battery_kwh']
        additional_pv_kw = params['additional_pv_kw']
        allocation_factor = params['allocation_factor']
        
        # Calculate the allocation of additional PV capacity
        type_a_capacity = min(additional_pv_kw * allocation_factor, 26.40)  # Capped at max capacity
        type_b_capacity = additional_pv_kw - type_a_capacity
        
        # Apply default battery power if not specified (0.5C rate)
        battery_kw = params.get('battery_kw', battery_kwh * 0.5)
        
        # Only simulate additional PV if there's any
        if additional_pv_kw > 0:
            # Create roof parameters for simulation
            additional_roof_params = []
            
            # Add Type A if capacity > 0
            if type_a_capacity > 0:
                type_a_params = objective_extra_pv.TYPE_A_ROOF_PARAMS.copy()
                type_a_params['system_capacity_kw'] = type_a_capacity
                additional_roof_params.append(type_a_params)
            
            # Add Type B if capacity > 0
            if type_b_capacity > 0:
                type_b_params = objective_extra_pv.TYPE_B_ROOF_PARAMS.copy()
                type_b_params['system_capacity_kw'] = type_b_capacity
                additional_roof_params.append(type_b_params)
            
            # Simulate additional PV generation
            additional_pv_profile = simulate_multi_year_pv(
                weather_files=objective_extra_pv.WEATHER_FILES,
                roof_params=additional_roof_params,
                repeats_per_file=10,
                start_years=objective_extra_pv.START_YEARS
            )
            
            # Combine existing and additional PV generation
            total_pv_kwh = pv_profile['simulated_kwh'] + additional_pv_profile['simulated_kwh']
            total_pv_profile = pd.DataFrame({'simulated_kwh': total_pv_kwh})
            
            # Update total PV capacity
            total_pv_capacity = objective_extra_pv.EXISTING_PV_CAPACITY + additional_pv_kw
        else:
            # No additional PV, use existing profile and capacity
            total_pv_profile = pv_profile
            total_pv_capacity = objective_extra_pv.EXISTING_PV_CAPACITY
        
        # Extract PV generation series
        if hasattr(total_pv_profile, 'columns'):
            gen = total_pv_profile['simulated_kwh']
        else:
            gen = total_pv_profile
        
        # Simulate battery dispatch with updated parameters
        dispatch_df, totals = simulate_battery_dispatch(
            pv_gen=gen,
            demand=demand_profile,
            battery_kwh=battery_kwh,
            battery_kw=battery_kw,
            roundtrip_eff=0.9,           # 90% round trip efficiency
            min_soc_pct=0.05,            # 95% depth of discharge (5% min SOC)
            annual_deg_rate=0.01,        # 1% degradation per year
            grid_emission_rate=0.81      # 0.81 kg CO2e/kWh
        )
        
        # Calculate PV capital costs
        pv_cost_per_kw = 1500.0  # Base cost per kW for PV
        pv_type_a_cost = type_a_capacity * pv_cost_per_kw  # Existing roof installation
        pv_type_b_cost = type_b_capacity * (pv_cost_per_kw * 1.15)  # 15% premium for optimal-angle installation
        pv_installation_cost = 1000.0 if additional_pv_kw > 0 else 0.0  # Fixed installation overhead
        total_pv_cost = pv_type_a_cost + pv_type_b_cost + pv_installation_cost
        
        # Compute financial metrics with updated parameters
        fin = compute_financials(
            totals,
            battery_kwh=battery_kwh,
            pv_kw=total_pv_capacity,                 
            pv_cost_per_kw=total_pv_cost/additional_pv_kw if additional_pv_kw > 0 else 0.0,  # Effective cost per kW
            pv_installation_cost=0.0,              # Already included in total_pv_cost
            battery_cost_per_kwh=None,             # Use formula: 977.54 * e^(-0.004*x) with $600 minimum
            battery_installation_cost_per_kwh=174.0,
            battery_power_ratio=0.5,               # Power rating as fraction of capacity
            pv_maintenance_per_kw_day=0.13,        # $0.13 per kW per day for PV
            battery_maintenance_per_kw_day=0.12,   # $0.12 per kW per day for battery
            discount_rate=0.07,
            baseline_import_cost=objective_extra_pv.BASE_IMPORT_COST,  # No PV, no battery cost
            baseline_pv_only_cost=objective_extra_pv.PV_ONLY_COST      # PV-only, no battery cost
        )
        
        # Return objectives to minimize: negative IRR, negative NPV
        irr = fin['irr'] or 0.0  # Handle None case
        npv = -fin['net_cost']   # Convert NPC to NPV
        
        return [-irr, -npv]  # Only return financial metrics
    
    # Configure objective module with global parameters
    objective_extra_pv.EXISTING_PV_CAPACITY = total_pv_capacity
    objective_extra_pv.WEATHER_FILES = weather_files
    objective_extra_pv.START_YEARS = start_years
    objective_extra_pv.BASE_IMPORT_COST = BASE_IMPORT_COST
    objective_extra_pv.PV_ONLY_COST = PV_ONLY_COST
    
    # Create an instance of the problem
    problem = BatteryPVOptimizationProblem()
    print("Optimization problem defined successfully")
    
except Exception as e:
    print(f"Error defining optimization problem: {e}")
    raise

Defining optimization problem with battery + additional PV (IRR and NPV only)...
Optimization problem defined successfully


In [7]:
# Cell 7: Run NSGA‑II optimization with battery + additional PV (IRR and NPV only)
print("Running NSGA‑II optimization with battery + additional PV (IRR and NPV only)...")
total_start_time = time.time()

# Create callback for tracking optimization progress with detailed stats
class BestSolutionCallback(Callback):
    def __init__(self):
        super().__init__()
        self.data = {
            "gen": [],
            "best_irr": [],
            "best_npv": [],
            "batt_irr": [],
            "batt_npv": [],
            "time_elapsed": []
        }
        self.start_time = time.time()
        self.last_print = self.start_time
    
    def notify(self, algorithm):
        gen = algorithm.n_gen
        self.data["gen"].append(gen)
        
        # Get current best objectives
        F = algorithm.pop.get("F")
        best_irr = -np.min(F[:, 0])  # Convert back from -IRR
        best_npv = -np.min(F[:, 1])  # Convert back from -NPV
        
        self.data["best_irr"].append(best_irr)
        self.data["best_npv"].append(best_npv)
        
        # Calculate time statistics
        elapsed = time.time() - self.start_time
        self.data["time_elapsed"].append(elapsed)
        
        # Calculate average time per generation
        avg_time_per_gen = elapsed / gen if gen > 0 else 0
        
        # Estimate time remaining
        n_generations = algorithm.termination.n_max_gen
        remaining_gens = n_generations - gen
        est_remaining_time = avg_time_per_gen * remaining_gens
        
        # Find battery sizes for best IRR and NPV
        X = algorithm.pop.get("X")
        best_irr_idx = np.argmin(F[:, 0])
        best_npv_idx = np.argmin(F[:, 1])
        
        best_irr_batt = X[best_irr_idx, 0]
        best_irr_pv = X[best_irr_idx, 1]
        best_npv_batt = X[best_npv_idx, 0]
        best_npv_pv = X[best_npv_idx, 1]
        
        # Store battery size for best IRR and NPV
        self.data["batt_irr"].append(best_irr_batt)
        self.data["batt_npv"].append(best_npv_batt)
        
        # Log progress every 5 generations or if more than 30 seconds passed since last print
        current_time = time.time()
        time_since_last_print = current_time - self.last_print
        
        if gen % 5 == 0 or gen == 1 or time_since_last_print > 30:
            print(f"Generation {gen:3d}/{n_generations}: " 
                  f"Best IRR = {best_irr*100:7.2f}% (Batt: {best_irr_batt:.1f} kWh, PV: {best_irr_pv:.1f} kW), " 
                  f"Best NPV = ${best_npv:10,.2f} (Batt: {best_npv_batt:.1f} kWh, PV: {best_npv_pv:.1f} kW), "
                  f"Time: {timedelta(seconds=int(elapsed))}, "
                  f"Est. remaining: {timedelta(seconds=int(est_remaining_time))}")
            self.last_print = current_time

callback = BestSolutionCallback()

# Population size and number of generations
pop_size = 40  # Changed from 60 to 40
n_generations = 50  # Changed from 70 to 50
print(f"Population size: {pop_size}, Generations: {n_generations}")
print(f"Total evaluations: {pop_size * n_generations}")

res = minimize(
    problem,
    NSGA2(pop_size=pop_size),
    get_termination("n_gen", n_generations),
    seed=42,
    verbose=False,
    callback=callback
)

# Extract & save Pareto front
solutions = res.X  # shape (n_solutions, 3)
battery_sizes = solutions[:, 0]
additional_pv = solutions[:, 1]
allocation_factors = solutions[:, 2]
pareto_F = res.F  # shape (n_solutions, 2) - now only IRR and NPV
irr_vals = -pareto_F[:, 0]  # Convert from -IRR to IRR (higher is better)
npv_vals = -pareto_F[:, 1]  # Convert from -NPV to NPV (higher is better)

df = pd.DataFrame({
    'battery_kwh': battery_sizes,
    'additional_pv_kw': additional_pv,
    'allocation_factor': allocation_factors,
    'type_a_capacity': additional_pv * allocation_factors,
    'type_b_capacity': additional_pv * (1 - allocation_factors),
    'IRR': irr_vals,
    'NPV': npv_vals
})

# Add calculated emissions for reference (but not used in optimization)
print("Calculating emissions for reference (not used in optimization)...")
emissions = []
for i in range(len(df)):
    params = {
        'battery_kwh': df.iloc[i]['battery_kwh'],
        'additional_pv_kw': df.iloc[i]['additional_pv_kw'],
        'allocation_factor': df.iloc[i]['allocation_factor']
    }
    # Use the original evaluate_solution to get emissions (third value)
    result = evaluate_solution(params, pv_profile, demand_profile)
    emissions.append(result[2])  # Emissions are the third value

df['emissions'] = emissions

# Total runtime
total_elapsed = time.time() - total_start_time
print(f"\nOptimization complete! Total runtime: {timedelta(seconds=int(total_elapsed))}")
print(f"Average time per evaluation: {total_elapsed/(pop_size * n_generations):.3f} seconds")

# Save results
(out_dir / 'pareto_solutions.csv').write_text(df.to_csv(index=False))
print(f"✅ Pareto front saved to {out_dir/'pareto_solutions.csv'}")

# Display best solutions for each objective
print("\nBest solutions found:")
print(f"Best IRR: {df['IRR'].max()*100:.2f}% with {df.loc[df['IRR'].idxmax(), 'battery_kwh']:.1f} kWh battery and {df.loc[df['IRR'].idxmax(), 'additional_pv_kw']:.1f} kW additional PV")
print(f"Best NPV: ${df['NPV'].max():,.2f} with {df.loc[df['NPV'].idxmax(), 'battery_kwh']:.1f} kWh battery and {df.loc[df['NPV'].idxmax(), 'additional_pv_kw']:.1f} kW additional PV")

# Create a scatter plot of the Pareto front
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['IRR']*100, df['NPV'], 
                     c=df['battery_kwh'], s=df['additional_pv_kw']*5, 
                     cmap='viridis', alpha=0.7)

plt.colorbar(scatter, label='Battery Size (kWh)')
plt.xlabel('IRR (%)')
plt.ylabel('NPV ($)')
plt.title('Pareto Front: IRR vs NPV')
plt.grid(True, linestyle='--', alpha=0.7)

# Annotate a few notable points
best_irr_idx = df['IRR'].idxmax()
best_npv_idx = df['NPV'].idxmax()

plt.annotate(f"Best IRR: {df['IRR'].max()*100:.1f}%\nBatt: {df.loc[best_irr_idx, 'battery_kwh']:.1f}kWh\nPV: {df.loc[best_irr_idx, 'additional_pv_kw']:.1f}kW",
             xy=(df.loc[best_irr_idx, 'IRR']*100, df.loc[best_irr_idx, 'NPV']),
             xytext=(10, -30), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2'))

plt.annotate(f"Best NPV: ${df['NPV'].max():,.0f}\nBatt: {df.loc[best_npv_idx, 'battery_kwh']:.1f}kWh\nPV: {df.loc[best_npv_idx, 'additional_pv_kw']:.1f}kW",
             xy=(df.loc[best_npv_idx, 'IRR']*100, df.loc[best_npv_idx, 'NPV']),
             xytext=(-70, 30), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2'))

plt.tight_layout()
plt.savefig(out_dir / 'pareto_front_irr_npv.png', dpi=300)
plt.close()

print(f"✅ Pareto front visualization saved to {out_dir/'pareto_front_irr_npv.png'}")

# Save convergence data
convergence_df = pd.DataFrame({
    'Generation': callback.data['gen'],
    'Best_IRR': [irr * 100 for irr in callback.data['best_irr']],
    'Best_NPV': callback.data['best_npv'],
    'Battery_IRR': callback.data['batt_irr'],
    'Battery_NPV': callback.data['batt_npv'],
    'Time_Elapsed': callback.data['time_elapsed']
})
convergence_df.to_csv(out_dir / 'convergence_data.csv', index=False)
print(f"✅ Convergence data saved to {out_dir/'convergence_data.csv'}")

# Create convergence plot
plt.figure(figsize=(12, 8))

# Create two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Plot IRR convergence
ax1.plot(callback.data['gen'], [irr * 100 for irr in callback.data['best_irr']], 'b-', linewidth=2)
ax1.set_ylabel('Best IRR (%)')
ax1.set_title('NSGA-II Convergence: IRR and NPV')
ax1.grid(True, linestyle='--', alpha=0.7)

# Plot NPV convergence
ax2.plot(callback.data['gen'], callback.data['best_npv'], 'g-', linewidth=2)
ax2.set_xlabel('Generation')
ax2.set_ylabel('Best NPV ($)')
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(out_dir / 'convergence_plot.png', dpi=300)
plt.close()

print(f"✅ Convergence plot saved to {out_dir/'convergence_plot.png'}")

Running NSGA‑II optimization with battery + additional PV (IRR and NPV only)...
Population size: 40, Generations: 50
Total evaluations: 2000
Generation   1/50: Best IRR =    3.59% (Batt: 7.0 kWh, PV: 14.6 kW), Best NPV = $ 39,495.22 (Batt: 15.7 kWh, PV: 25.4 kW), Time: 0:01:01, Est. remaining: 0:50:17
Generation   2/50: Best IRR =    3.65% (Batt: 3.1 kWh, PV: 14.6 kW), Best NPV = $ 39,495.22 (Batt: 15.7 kWh, PV: 25.4 kW), Time: 0:02:06, Est. remaining: 0:50:39
Generation   3/50: Best IRR =    3.76% (Batt: 3.4 kWh, PV: 13.6 kW), Best NPV = $ 39,495.22 (Batt: 15.7 kWh, PV: 25.4 kW), Time: 0:03:12, Est. remaining: 0:50:20
Generation   4/50: Best IRR =    3.88% (Batt: 0.2 kWh, PV: 12.7 kW), Best NPV = $ 40,193.50 (Batt: 15.7 kWh, PV: 29.6 kW), Time: 0:04:16, Est. remaining: 0:49:14
Generation   5/50: Best IRR =    3.88% (Batt: 0.2 kWh, PV: 12.7 kW), Best NPV = $ 40,193.50 (Batt: 15.7 kWh, PV: 29.6 kW), Time: 0:05:22, Est. remaining: 0:48:18
Generation   6/50: Best IRR =    3.91% (Batt: 0.2

<Figure size 1200x800 with 0 Axes>

In [8]:
# Cell 8: Run NSGA‑II optimization with battery + additional PV
print("Running NSGA‑II optimization with battery + additional PV…")
total_start_time = time.time()
callback = BestSolutionCallback()
# Population size and number of generations
pop_size = 60  # Increased for 3-variable problem
n_generations = 70  # Increased for better convergence
print(f"Population size: {pop_size}, Generations: {n_generations}")
print(f"Total evaluations: {pop_size * n_generations}")
res = minimize(
    problem,
    NSGA2(pop_size=pop_size),
    get_termination("n_gen", n_generations),
    seed=42,
    verbose=False,
    callback=callback
)
# Extract & save Pareto front
solutions = res.X  # shape (n_solutions, 3)
battery_sizes = solutions[:, 0]
additional_pv = solutions[:, 1]
allocation_factors = solutions[:, 2]

pareto_F = res.F  # shape (n_solutions, 3)
irr_vals = -pareto_F[:, 0]  # Convert from -IRR to IRR (higher is better)
npv_vals = -pareto_F[:, 1]  # Convert from -NPV to NPV (higher is better)
emission_vals = pareto_F[:, 2]  # Emissions (lower is better)

df = pd.DataFrame({
    'battery_kwh': battery_sizes,
    'additional_pv_kw': additional_pv,
    'allocation_factor': allocation_factors,
    'type_a_capacity': additional_pv * allocation_factors,
    'type_b_capacity': additional_pv * (1 - allocation_factors),
    'IRR': irr_vals,
    'NPV': npv_vals,
    'emissions': emission_vals
})

# Total runtime
total_elapsed = time.time() - total_start_time
print(f"\nOptimization complete! Total runtime: {timedelta(seconds=int(total_elapsed))}")
print(f"Average time per evaluation: {total_elapsed/(pop_size * n_generations):.3f} seconds")
(out_dir / 'pareto_solutions.csv').write_text(df.to_csv(index=False))
print(f"✅ Pareto front saved to {out_dir/'pareto_solutions.csv'}")

# Display best solutions for each objective
print("\nBest solutions found:")
print(f"Best IRR: {df['IRR'].max()*100:.2f}% with {df.loc[df['IRR'].idxmax(), 'battery_kwh']:.1f} kWh battery and {df.loc[df['IRR'].idxmax(), 'additional_pv_kw']:.1f} kW additional PV")
print(f"Best NPV: ${df['NPV'].max():,.2f} with {df.loc[df['NPV'].idxmax(), 'battery_kwh']:.1f} kWh battery and {df.loc[df['NPV'].idxmax(), 'additional_pv_kw']:.1f} kW additional PV")
print(f"Lowest Emissions: {df['emissions'].min():,.2f} kg CO2e with {df.loc[df['emissions'].idxmin(), 'battery_kwh']:.1f} kWh battery and {df.loc[df['emissions'].idxmin(), 'additional_pv_kw']:.1f} kW additional PV")

Running NSGA‑II optimization with battery + additional PV…
Population size: 60, Generations: 70
Total evaluations: 4200
Generation   1/70: Best IRR =    3.68% (Batt: 6.0 kWh, PV: 16.9 kW), Best NPV = $ 39,951.54 (Batt: 16.2 kWh, PV: 25.9 kW), Time: 0:01:27, Est. remaining: 1:40:16
Generation   2/70: Best IRR =    3.69% (Batt: 6.0 kWh, PV: 11.8 kW), Best NPV = $ 39,951.54 (Batt: 16.2 kWh, PV: 25.9 kW), Time: 0:02:54, Est. remaining: 1:38:55
Generation   3/70: Best IRR =    3.72% (Batt: 5.3 kWh, PV: 11.8 kW), Best NPV = $ 40,327.98 (Batt: 18.0 kWh, PV: 28.6 kW), Time: 0:04:22, Est. remaining: 1:37:41
Generation   4/70: Best IRR =    3.72% (Batt: 5.3 kWh, PV: 11.8 kW), Best NPV = $ 40,502.54 (Batt: 18.1 kWh, PV: 28.9 kW), Time: 0:05:49, Est. remaining: 1:36:10
Generation   5/70: Best IRR =    3.72% (Batt: 5.3 kWh, PV: 12.4 kW), Best NPV = $ 40,525.02 (Batt: 18.1 kWh, PV: 29.4 kW), Time: 0:07:17, Est. remaining: 1:34:53
Generation   6/70: Best IRR =    3.83% (Batt: 2.0 kWh, PV: 11.8 kW), B

IndexError: index 2 is out of bounds for axis 1 with size 2

In [1]:
# Cell 9: Save Full 30-Year Time Series and Data Dictionary
# Run the simulation again for the best solution to get detailed results
try:
    # Try to find best IRR solution
    best_irr_idx = df['IRR'].idxmax()
    
    # Check if the index is valid
    if pd.isna(best_irr_idx):
        # If all IRR values are NaN, use the solution with the best NPV
        print("No valid IRR found. Using solution with best NPV instead.")
        best_npv_idx = df['NPV'].idxmax()  # Since NPV is now positive (we negated NPC)
        best_battery_kwh = df.loc[best_npv_idx, 'battery_kwh']
    else:
        # If a valid IRR exists, use that solution
        best_battery_kwh = df.loc[best_irr_idx, 'battery_kwh']
except Exception as e:
    # Fallback if errors occur: find solution with minimum NPC/maximum NPV
    print(f"Error finding optimal solution based on IRR: {str(e)}")
    print("Using solution with best NPV instead.")
    best_npv_idx = df['NPV'].idxmax()
    best_battery_kwh = df.loc[best_npv_idx, 'battery_kwh']

print(f"\nSimulating best solution for detailed 30-year time series...")
print(f"Selected battery size: {best_battery_kwh:.2f} kWh")
start_time = time.time()

# If best battery size is 0, use a small non-zero value for demonstration
if best_battery_kwh == 0:
    print("Best battery size is 0 kWh. Using 1 kWh for demonstration purposes.")
    analysis_battery_kwh = 1.0
else:
    analysis_battery_kwh = best_battery_kwh

# Calculate battery cost using formula
if analysis_battery_kwh <= 0:
    battery_cost_per_kwh = 0.0  # No cost for 0 kWh battery
else:
    battery_cost_per_kwh = 977.54 * np.exp(-0.004 * analysis_battery_kwh)
    battery_cost_per_kwh = max(600.0, battery_cost_per_kwh)  # Minimum $600/kWh
print(f"Battery cost: ${battery_cost_per_kwh:.2f} per kWh")

# Run simulation with selected battery size and updated parameters
best_disp, best_tots = simulate_battery_dispatch(
    pv_profile['simulated_kwh'],
    demand_profile,
    battery_kwh=analysis_battery_kwh,
    battery_kw=analysis_battery_kwh * 0.5,  # 0.5C rate
    roundtrip_eff=0.9,                      # 90% round trip efficiency
    min_soc_pct=0.05,                       # 95% depth of discharge
    annual_deg_rate=0.01,                   # 1% annual degradation
    grid_emission_rate=0.81                 # 0.81 kg CO2e/kWh
)
elapsed = time.time() - start_time
print(f"Detailed simulation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")

# Report battery cycles and degradation
print(f"Battery cycles over 30 years: {best_tots['battery_cycles']:.1f}")
print(f"Final battery degradation: {best_tots['final_degradation_pct']:.1f}%")

# Calculate financial metrics for the best solution in detail
best_fin = compute_financials(
    best_tots,
    battery_kwh=analysis_battery_kwh,
    pv_kw=total_pv_capacity,
    pv_cost_per_kw=0.0,                    # Existing PV system
    pv_installation_cost=0.0,              # Existing PV system
    battery_cost_per_kwh=battery_cost_per_kwh,  # Use calculated cost
    battery_installation_cost_per_kwh=174.0,
    battery_power_ratio=0.5,               # Power rating as fraction of capacity
    pv_maintenance_per_kw_day=0.13,        # $0.13 per kW per day for PV
    battery_maintenance_per_kw_day=0.12,   # $0.12 per kW per day for battery
    discount_rate=0.07,
    baseline_import_cost=BASE_IMPORT_COST
)

# Save full 30-year time series to CSV
print(f"Saving full 30-year time series data... ({len(best_disp)} rows)")
best_disp.to_csv(out_dir / "best_irr_30yr_timeseries.csv")
print(f"Saved 30-year time series to {out_dir / 'best_irr_30yr_timeseries.csv'}")

# Create a data dictionary with variable definitions and store as CSV
data_dict = pd.DataFrame([
    {"Variable": "pv_gen", "Description": "PV generation in kWh for each half-hour", "Units": "kWh"},
    {"Variable": "demand", "Description": "Electricity demand in kWh for each half-hour", "Units": "kWh"},
    {"Variable": "pv_used", "Description": "PV generation used directly to satisfy demand", "Units": "kWh"},
    {"Variable": "battery_charge", "Description": "Energy used to charge the battery", "Units": "kWh"},
    {"Variable": "battery_discharge", "Description": "Energy discharged from the battery", "Units": "kWh"},
    {"Variable": "battery_soc", "Description": "Battery state of charge", "Units": "kWh"},
    {"Variable": "pv_export", "Description": "Excess PV generation exported to the grid", "Units": "kWh"},
    {"Variable": "grid_import_peak", "Description": "Electricity imported during peak hours", "Units": "kWh"},
    {"Variable": "grid_import_offpeak", "Description": "Electricity imported during off-peak hours", "Units": "kWh"}
])
data_dict.to_csv(out_dir / "data_dictionary.csv", index=False)
print(f"Saved data dictionary to {out_dir / 'data_dictionary.csv'}")

# Create comprehensive summary
summary_df = pd.DataFrame({
    "Parameter": ["Optimal Battery Size", "PV Size", "Battery Cost", "IRR", "NPV", "Renewable Fraction", "Battery Cycles", "Final Degradation"],
    "Value": [
        best_battery_kwh,  # The actual optimal size (may be 0)
        total_pv_capacity, 
        battery_cost_per_kwh,
        best_fin["irr"] if best_fin["irr"] is not None else "Not calculable",
        best_fin["net_cost"], 
        best_tots["renewable_fraction"],
        best_tots["battery_cycles"],
        best_tots["final_degradation_pct"]
    ],
    "Units": ["kWh", "kW", "$/kWh", "fraction", "$", "fraction", "cycles", "%"],
    "Description": [
        "Optimal battery size (0 = no battery is optimal)", 
        "Fixed PV system capacity", 
        f"Battery cost using formula: 977.54*e^(-0.004*x) with $600 minimum",
        "Internal Rate of Return (may be undefined)", 
        "Net Present Cost", 
        "Fraction of demand met by renewable sources",
        "Total battery charge/discharge cycles over 30 years",
        "Final battery capacity degradation percentage"
    ]
})
summary_df.to_csv(out_dir / "optimization_summary.csv", index=False)
print(f"Saved optimization summary to {out_dir / 'optimization_summary.csv'}")

# Note about the results
if best_battery_kwh == 0:
    print("\nIMPORTANT NOTE: The optimization found that a 0 kWh battery (no battery) is optimal.")
    print("This means batteries are not economically justified with current parameters.")
    print("Consider adjusting electricity rates, battery costs, or maintenance costs to create")
    print("scenarios where batteries provide positive economic value.")
    print("\nFor demonstration purposes, the analysis was conducted with a 1 kWh battery.")

print("\nAnalysis complete!")

Error finding optimal solution based on IRR: name 'df' is not defined
Using solution with best NPV instead.


NameError: name 'df' is not defined

In [25]:
# Cell 10: Create daily profile plots and export final results
print("Creating daily profile plots...")
# Summer day
import matplotlib.pyplot as plt
summer_date = pd.Timestamp(f"{demand_profile.index[0].year}-01-15")
summer_mask = (best_disp.index.date == summer_date.date())
summer_data = best_disp[summer_mask]

plt.figure(figsize=(12, 6))
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['pv_gen'], 'orange', label='PV Generation')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['demand'], 'blue', label='Demand')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['battery_discharge'], 'green', label='Battery Discharge')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['grid_import_peak'] + summer_data['grid_import_offpeak'], 'red', label='Grid Import')
plt.fill_between(summer_data.index.strftime('%H:%M'), summer_data['battery_soc'], alpha=0.3, color='green', label='Battery SOC')
plt.xticks(rotation=90)
plt.title(f'Summer Day Profile: {summer_date.strftime("%B %d")}')
plt.xlabel('Time of Day')
plt.ylabel('Energy (kWh)')
plt.legend()
plt.tight_layout()
plt.savefig(out_dir / "summer_day_profile.png", dpi=300)
plt.close()

# Winter day
winter_date = pd.Timestamp(f"{demand_profile.index[0].year}-07-15")
winter_mask = (best_disp.index.date == winter_date.date())
winter_data = best_disp[winter_mask]

plt.figure(figsize=(12, 6))
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['pv_gen'], 'orange', label='PV Generation')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['demand'], 'blue', label='Demand')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['battery_discharge'], 'green', label='Battery Discharge')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['grid_import_peak'] + winter_data['grid_import_offpeak'], 'red', label='Grid Import')
plt.fill_between(winter_data.index.strftime('%H:%M'), winter_data['battery_soc'], alpha=0.3, color='green', label='Battery SOC')
plt.xticks(rotation=90)
plt.title(f'Winter Day Profile: {winter_date.strftime("%B %d")}')
plt.xlabel('Time of Day')
plt.ylabel('Energy (kWh)')
plt.legend()
plt.tight_layout()
plt.savefig(out_dir / "winter_day_profile.png", dpi=300)
plt.close()

print("Profile plots created and saved to:", out_dir)

# Export all remaining data
# 1) Roof configuration
pd.DataFrame(roof_params).to_csv(out_dir / 'roof_config.csv', index=False)

# 2) Weather files list
pd.DataFrame({'weather_file': weather_files}).to_csv(out_dir / 'weather_files.csv', index=False)

# 3) Best solution details as text file
with open(out_dir / "best_solution_details.txt", "w") as f:
    f.write(f"Best Solution Details:\n")
    f.write(f"PV Size: {total_pv_capacity:.1f} kW\n")
    f.write(f"Battery Size: {best_battery_kwh:.1f} kWh\n")
    
    # Calculate battery cost for text file
    if analysis_battery_kwh <= 0:
        battery_cost = 0.0
    else:
        battery_cost = 977.54 * np.exp(-0.004 * analysis_battery_kwh)
        battery_cost = max(600.0, battery_cost)
    f.write(f"Battery Cost: ${battery_cost:.2f} per kWh\n")
    
    f.write(f"IRR: {best_fin['irr']:.2%}\n") if best_fin['irr'] is not None else f.write(f"IRR: Not calculable\n")
    f.write(f"Net Present Cost: ${best_fin['net_cost']:.2f}\n")
    f.write(f"\nSystem Configuration:\n")
    f.write(f"PV Tilt: {roof_params[0]['tilt']}°\n")
    f.write(f"PV Azimuth: {roof_params[0]['azimuth']}°\n")
    f.write(f"PV Shading: {roof_params[0]['shading']}%\n")
    f.write(f"\nSimulation Parameters:\n")
    f.write(f"PV Cost: $0 (existing system)\n")
    f.write(f"Battery Cost: Formula: 977.54*e^(-0.004*x) with $600 minimum\n")
    f.write(f"Battery Installation Cost: $174/kWh\n")
    f.write(f"PV Maintenance: $0.13/kW/day\n")
    f.write(f"Battery Maintenance: $0.12/kW/day\n")
    f.write(f"Discount Rate: 7%\n")
    f.write(f"Battery Degradation: 1%/year\n")
    f.write(f"Battery Depth of Discharge: 95%\n")
    f.write(f"Battery Round Trip Efficiency: 90%\n")
    f.write(f"Battery C-Rate: 0.5C\n")
    f.write(f"Grid Emission Rate: 0.81 kg CO2e/kWh\n")
    f.write(f"\nEnergy Summary:\n")
    f.write(f"Total Demand: {best_tots['total_demand']:.2f} kWh\n")
    f.write(f"Total PV Used Directly: {best_tots['total_pv_used']:.2f} kWh\n")
    f.write(f"Total Battery Discharge: {best_tots['total_battery_discharge']:.2f} kWh\n")
    f.write(f"Total Grid Import (Peak): {best_tots['total_grid_import_peak']:.2f} kWh\n")
    f.write(f"Total Grid Import (Off-Peak): {best_tots['total_grid_import_offpeak']:.2f} kWh\n")
    f.write(f"Total PV Export: {best_tots['total_pv_export']:.2f} kWh\n")
    f.write(f"Renewable Fraction: {best_tots['renewable_fraction']:.2%}\n")
    f.write(f"Self-Consumption Rate: {best_tots['self_consumption_rate']:.2%}\n")
    f.write(f"Battery Cycles: {best_tots['battery_cycles']:.1f}\n")
    f.write(f"Final Battery Degradation: {best_tots['final_degradation_pct']:.1f}%\n")
# 4) Financial summary
pd.DataFrame.from_dict(best_fin, orient='index', columns=['value']).to_csv(out_dir / 'financial_summary.csv')

# 5) Dispatch totals  
pd.DataFrame.from_dict(best_tots, orient='index', columns=['value']).to_csv(out_dir / 'dispatch_totals.csv')

# 6) Convergence data
convergence_df = pd.DataFrame({
    'Generation': callback.data['gen'],
    'Best_IRR': [irr * 100 for irr in callback.data['best_irr']],
    'Best_NPV': callback.data['best_npv'],
    'Battery_IRR': callback.data['batt_irr'],
    'Battery_NPV': callback.data['batt_npv'],
    'Time_Elapsed': callback.data['time_elapsed']
})
convergence_df.to_csv(out_dir / 'convergence_data.csv', index=False)

print(f"\n✅ All outputs successfully saved to {out_dir}")
print("NSGA-II optimization complete!")

Creating daily profile plots...
Profile plots created and saved to: /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga_outputs/003


KeyError: 'best_npc'

In [ ]:
# Cell 11: Calculate and display electricity cost savings with battery
print("Calculating electricity cost comparison...")
try:
    # Get the best IRR solution
    best_irr_idx = df['IRR'].idxmax()
    best_battery = df.loc[best_irr_idx, 'battery_kwh']
    
    # Calculate for first year only (17520 points)
    # 1) Annual cost without PV or battery (baseline)
    annual_demand = demand_profile[:17520].sum()  
    annual_peak_demand = 0
    annual_offpeak_demand = 0
    
    # Calculate peak/offpeak split based on TOD tariff
    print("Calculating peak/off-peak demand split...")
    for idx, val in demand_profile[:17520].items():
        h = idx.hour
        m = idx.month
        # Summer (Oct-Mar): 2pm-8pm, Winter (Apr-Sep): 3pm-9pm
        if (m < 4 or m > 9):  # Summer
            is_peak = (h >= 14 and h < 20)
        else:  # Winter
            is_peak = (h >= 15 and h < 21)
        
        if is_peak:
            annual_peak_demand += val
        else:
            annual_offpeak_demand += val
    
    baseline_cost = annual_peak_demand * 0.39710 + annual_offpeak_demand * 0.13530
    
    # 2) Annual cost with PV but no battery
    print("Calculating costs with PV only...")
    disp_pv_only, tots_pv_only = simulate_battery_dispatch(
        pv_gen=pv_profile['simulated_kwh'][:17520],
        demand=demand_profile[:17520],
        battery_kwh=0.0
    )
    
    pv_only_cost = (tots_pv_only['total_grid_import_peak'] * 0.39710 + 
                    tots_pv_only['total_grid_import_offpeak'] * 0.13530 - 
                    tots_pv_only['total_pv_export'] * 0.033)
    
    # 3) Annual cost with PV and battery
    print(f"Calculating costs with PV and {best_battery:.1f} kWh battery...")
    disp_pv_batt, tots_pv_batt = simulate_battery_dispatch(
        pv_gen=pv_profile['simulated_kwh'][:17520],
        demand=demand_profile[:17520],
        battery_kwh=best_battery,
        battery_kw=best_battery * 0.5,
        roundtrip_eff=0.9,
        min_soc_pct=0.2,
        annual_deg_rate=0.01
    )
    
    pv_batt_cost = (tots_pv_batt['total_grid_import_peak'] * 0.39710 + 
                    tots_pv_batt['total_grid_import_offpeak'] * 0.13530 - 
                    tots_pv_batt['total_pv_export'] * 0.033)
    
    # Calculate savings
    savings_pv_only = baseline_cost - pv_only_cost
    savings_pv_batt = baseline_cost - pv_batt_cost
    additional_savings = pv_only_cost - pv_batt_cost
    
    # Save results to file
    savings_df = pd.DataFrame({
        'Scenario': ['Baseline (no PV/battery)', 'PV Only', f'PV + {best_battery:.1f}kWh Battery'],
        'Annual Cost ($)': [baseline_cost, pv_only_cost, pv_batt_cost],
        'Savings vs Baseline ($)': [0, savings_pv_only, savings_pv_batt],
        'Electricity Cost Reduction (%)': [0, savings_pv_only/baseline_cost*100, savings_pv_batt/baseline_cost*100]
    })
    savings_df.to_csv(out_dir / 'annual_electricity_savings.csv', index=False)
    
    # Print results
    print("\n=== Annual Electricity Cost Comparison ===")
    print(f"Baseline (no PV, no battery): ${baseline_cost:.2f}")
    print(f"With PV only:                 ${pv_only_cost:.2f}")
    print(f"With PV + {best_battery:.1f}kWh battery:    ${pv_batt_cost:.2f}")
    print("\n=== Annual Savings ===")
    print(f"Savings from PV only:            ${savings_pv_only:.2f}")
    print(f"Savings from PV + battery:       ${savings_pv_batt:.2f}")
    print(f"Additional savings from battery:  ${additional_savings:.2f}")
    print(f"This represents a {additional_savings/pv_only_cost*100:.1f}% reduction in electricity costs compared to PV-only")
    
except Exception as e:
    print(f"Error calculating electricity costs: {e}")